<a href="https://colab.research.google.com/github/Birsana/devito/blob/master/devitoCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np

import shutil


TensorFlow version is  1.14.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:

!unzip '/content/drive/My Drive/devito.zip'

Archive:  /content/drive/My Drive/devito.zip
   creating: devito/
  inflating: devito/QA_Danny_DeVito_008.jpg  
   creating: __MACOSX/
   creating: __MACOSX/devito/
  inflating: __MACOSX/devito/._QA_Danny_DeVito_008.jpg  
  inflating: devito/6GSF2WTIOZA6HBIXN5OJSX2KZY.jpg  
  inflating: __MACOSX/devito/._6GSF2WTIOZA6HBIXN5OJSX2KZY.jpg  
  inflating: devito/GettyImages_519768870_1_920x613.jpg  
  inflating: __MACOSX/devito/._GettyImages_519768870_1_920x613.jpg  
  inflating: devito/160123_stern_devito_tease_iy4lx3.jpg  
  inflating: __MACOSX/devito/._160123_stern_devito_tease_iy4lx3.jpg  
  inflating: devito/Jumanji_3_Cast_Danny_Devito.jpg  
  inflating: __MACOSX/devito/._Jumanji_3_Cast_Danny_Devito.jpg  
  inflating: devito/sw_28032019_danny_devito_large.jpg  
  inflating: __MACOSX/devito/._sw_28032019_danny_devito_large.jpg  
  inflating: devito/devito (6).jpg   
  inflating: __MACOSX/devito/._devito (6).jpg  
  inflating: devito/1HYTZ5G_002_lt.jpg  
  inflating: __MACOSX/devito/._1HY

In [4]:

path = "/content/devito"
files = os.listdir(path)
filesCopy = files
print(len(files))


331


In [0]:
os.mkdir("train_dir")
os.mkdir("validation_dir")

In [0]:
counter = 1
for file in files:
  if counter < 277:
    shutil.move("/content/devito/%s" % (file), "/content/train_dir")
  else:
    shutil.move("/content/devito/%s" % (file), "/content/validation_dir")
  counter += 1

In [7]:
check1 = os.listdir("/content/train_dir")
check2 = os.listdir("/content/validation_dir")

print(len(check1))
print(len(check2))

276
55


In [0]:
os.mkdir("train")
os.mkdir("val")




In [9]:
shutil.move("/content/train_dir", "/content/train")

shutil.move("/content/validation_dir", "/content/val")

'/content/val/validation_dir'

In [10]:
train_dir = "/content/train"
validation_dir = "/content/val"

image_size = 160
batch_size = 32
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
                train_dir, 
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, 
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')


Found 275 images belonging to 1 classes.
Found 55 images belonging to 1 classes.


In [11]:
IMG_SHAPE = (image_size, image_size, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

W0903 07:19:16.366136 139695582193536 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


9412608/9406464 [==============================] - 1s 0us/step


In [0]:
base_model.trainable = False

In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [0]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalMaxPooling2D(),  #TRY AVERAGE TOO
  keras.layers.Dense(1, activation='sigmoid')
])

In [15]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

W0903 07:19:28.289220 139695582193536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
epochs = 20
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

Epoch 1/20
8/8 [==============================] - 9s 1s/step - loss: 0.4198 - acc: 0.8354 - val_loss: 0.3112 - val_acc: 0.8438
Epoch 2/20
8/8 [==============================] - 5s 584ms/step - loss: 0.1464 - acc: 0.9342 - val_loss: 0.1507 - val_acc: 0.9375
Epoch 3/20
8/8 [==============================] - 5s 582ms/step - loss: 0.0588 - acc: 0.9835 - val_loss: 0.0883 - val_acc: 0.9688
Epoch 4/20
8/8 [==============================] - 4s 541ms/step - loss: 0.0396 - acc: 0.9877 - val_loss: 0.0524 - val_acc: 0.9688
Epoch 5/20
8/8 [==============================] - 5s 565ms/step - loss: 0.0303 - acc: 0.9883 - val_loss: 0.0273 - val_acc: 1.0000
Epoch 6/20
8/8 [==============================] - 4s 515ms/step - loss: 0.0058 - acc: 1.0000 - val_loss: 0.0207 - val_acc: 1.0000
Epoch 7/20
8/8 [==============================] - 4s 528ms/step - loss: 0.0092 - acc: 0.9959 - val_loss: 0.0125 - val_acc: 1.0000
Epoch 8/20
8/8 [==============================] - 6s 699ms/step - loss: 0.0030 - acc: 1.0000 